In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

# Cargo CSV

In [2]:
test_final = pd.read_csv('../../data/test_final_100k.csv')
test_final = test_final.drop(columns='id')

In [3]:
postulantes_educacion = pd.read_csv('../../data_sin_duplicados/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.drop(columns='Unnamed: 0')
postulantes_genero_y_edad = pd.read_csv('../../data_sin_duplicados/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='Unnamed: 0')
vistas = pd.read_csv('../../data_sin_duplicados/fiuba_3_vistas.csv')
vistas = vistas.drop(columns='Unnamed: 0')
postulaciones = pd.read_csv('../../data_sin_duplicados/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.drop(columns='Unnamed: 0')
avisos_online = pd.read_csv('../../data_sin_duplicados/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.drop(columns='Unnamed: 0')
avisos_detalle = pd.read_csv('../../data_sin_duplicados/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.drop(columns='Unnamed: 0')

# Procesamiento de datos

##### Agrego edad

In [4]:
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [5]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

##### Separo datos

In [6]:
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [7]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

##### Genero

In [8]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15707
0.0               6
Name: sexo, dtype: int64

In [9]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [10]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9635
0.0                1
Name: sexo, dtype: int64

In [11]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

##### Edad (cont.)

In [12]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [13]:
gente_no_postulada.isna().sum()

idpostulante       0
fechanacimiento    0
sexo               0
edad               0
dtype: int64

In [14]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [15]:
gente_postulada.isna().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad                   0
dtype: int64

# Funciones

In [16]:
# Fijarse de balancear el sample con 50% de postulados y no postulados
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [17]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop(columns='_merge')
    return df_return

In [18]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [19]:
def genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    df_return = df_return.drop(columns='fechanacimiento')
    return df_return

# Gente postulada

In [20]:
tupla_postulada = postulaciones.drop(columns='fechapostulacion')
tupla_postulada['y'] = 1
tupla_postulada = genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [21]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112257047,NM5M,1,48,1,0,0
1,1111920714,NM5M,1,48,1,0,0
2,1112346945,NM5M,1,48,1,0,0
3,1112345547,NM5M,1,48,1,0,0
4,1112237522,5awk,1,56,1,0,0


# Gente no postulada

In [24]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,100)
sample['y'] = 0
sample = genero_edad(sample,gente_no_postulada)
sample = oneHotEncoding(sample,'sexo')

In [25]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55,0,1,0
1,1112390041,ZDlz5E1,0,21,1,0,0
2,1112390041,a0N4q,0,43,1,0,0
3,1112390041,ep41j9,0,32,0,1,0
4,1112390041,awOL1m,0,34,1,0,0


# Test Final

In [27]:
test_final = genero_edad(test_final,gente_postulada.append(gente_no_postulada))

In [28]:
# Veo si hay datos nulos
test_final.isna().sum()

idaviso            0
idpostulante       0
sexo            2278
edad            2278
dtype: int64

In [29]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isna(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [30]:
test_final.isna().sum()

idaviso         0
idpostulante    0
sexo            0
edad            0
dtype: int64

In [31]:
test_final.head()

,idaviso,idpostulante,sexo,edad
0,739260,6M9ZQR,FEM,42.0
1,739260,6v1xdL,MASC,31.0
2,739260,ezRKm9,FEM,36.0
3,758580,1Q35ej,MASC,69.0
4,758580,EAN4J6,FEM,32.0


In [32]:
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

In [33]:
test_final.head()

,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,739260,6M9ZQR,42,1,0,0
1,739260,6v1xdL,31,0,1,0
2,739260,ezRKm9,36,1,0,0
3,758580,1Q35ej,69,0,1,0
4,758580,EAN4J6,32,1,0,0


# ML

In [34]:
# Metodo para calcular el error de la competencia
from sklearn.metrics import roc_auc_score

In [35]:
sample = sample.append(tupla_postulada.sample(n=1000000,random_state=0))

In [36]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55,0,1,0
1,1112390041,ZDlz5E1,0,21,1,0,0
2,1112390041,a0N4q,0,43,1,0,0
3,1112390041,ep41j9,0,32,0,1,0
4,1112390041,awOL1m,0,34,1,0,0


In [37]:
sample.tail()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
4019134,1112457433,RzMAmXo,1,24,1,0,0
1441298,1112306596,ekOJ0L9,1,31,0,1,0
1290650,1112262555,PmJExxE,1,29,1,0,0
4598828,1112296411,4rPXrkM,1,25,0,1,0
6441257,1112355579,xkPYo6z,1,42,0,1,0


In [38]:
sample = sample.drop(columns=['idaviso','idpostulante'])

In [39]:
len(sample)

2000000

In [40]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA
0,0,55,0,1,0
1,0,21,1,0,0
2,0,43,1,0,0
3,0,32,0,1,0
4,0,34,1,0,0


In [41]:
# Divido el set
X_train, X_test, y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

# Perceptron

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
model = Perceptron()
model.fit(X_train,y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# KNN

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#model = KNeighborsClassifier(n_neighbors=3)
#model.fit(X_train,y_train)
#predict = model.predict(X_test)

In [ ]:
#roc_auc_score(y_test,predict)

# Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
model = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [44]:
roc_auc_score(y_test,predict)

0.580524735289901

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier(n_estimators=150)
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)